In [ ]:
%load_ext autoreload
%autoreload 2

from models.vae import OsuReplayVAE
from models.rnn import OsuReplayRNN

In [2]:
RNN_PATH = ".trained/rnn_most_recent.pt"
rnn = OsuReplayRNN.load(RNN_PATH)

Position model compiled with torch.compile
RNN Model initialized on cuda (noise_std=0.0)
Position model parameters: 227650
RNN model loaded from .trained/rnn_most_recent.pt


In [3]:
VAE_PATH = ".trained/vae_most_recent.pt"
vae = OsuReplayVAE.load(VAE_PATH)

VAE models compiled..
VAE Models initialized on cuda
Encoder parameters: 236864
Decoder parameters: 260418
Beta: 0.0001
VAE models loaded from .trained/vae_most_recent.pt


In [4]:
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.dataset as dataset
import torch
import numpy as np

test_name = 'ringyou'
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'
test_mods = 0

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

Turning beatmaps into time series data: 100%|██████████| 1/1 [00:00<00:00,  3.83it/s]


In [5]:
model = vae

# generate 10 plays
NUM_PLAYS = 10

# get an array of [[x,y]] for each play
plays = [np.concatenate(model.generate(data)) for _ in range(NUM_PLAYS)]

In [6]:
# get the mean absolute error for two plays
def get_mae(play1, play2):
    return np.mean(np.abs(play1 - play2))

avg_mae = np.average([get_mae(plays[i], plays[i+1]) for i in range(NUM_PLAYS-1)])

In [7]:
avg_mae

np.float32(0.005260035)

In [8]:
# preview a replay
from osu.preview.preview import preview_replay_raw

replay_data = np.pad(plays[0], ((0, 0), (0, 2)), mode='constant', constant_values=0)

preview_replay_raw(replay_data, test_map_path, test_mods, test_song)

E:\Projects\osu\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# analyze stuff about the data
import numpy as np

xs = np.load(f'.datasets/xs_7500_07-30_22-38-46.npy')
ys = np.load(f'.datasets/ys_7500_07-30_22-38-46.npy')

In [1]:
# find which player appears the most
import osu.dataset as dataset
mapping = dataset.replay_mapping_from_cache(limit=20000)

Xlib.xauth: warning, no xauthority details available
Xlib.xauth: warning, no xauthority details available
osu! path: None/../Local/osu!


Loading replay metadata from .data/replays:  55%|█████▍    | 10922/20000 [00:04<00:04, 2220.95it/s]
Loading objects from .data/: 0it [00:00, ?it/s]


In [ ]:
players = [play.player for play in mapping['replay']]

player_counts = {}

for player in players:
    player_counts[player] = player_counts.get(player, 0) + 1
    
sorted_items = sorted(player_counts.items(), key=lambda item: item[1], reverse=True)

sorted_items

In [10]:
import polars as pl
df = pl.read_csv('all_scores.csv')

In [28]:
df_filtered = df.filter(
    (pl.col('accuracy') > 0.9) &
    (pl.col('user_id') == 7562902)
)

In [30]:
df_filtered[1]

id,user_id,beatmap_id,has_replay,preserve,ranked,rank,passed,accuracy,max_combo,total_score,ended_at,pp,beatmapset_id
i64,i64,i64,i64,i64,i64,str,i64,f64,i64,i64,str,str,i64
546969065,7562902,768617,1,1,1,"""A""",1,0.965155,851,943092,"""2018-01-05 23:05:08""","""107.66000366""",341933
